# Reproducing Perséphone results

## General instructions

- For each figure, the notebook contains commands to generate and plot the results.
- Start a terminal on the docker instance with `docker exec -it CONTAINER_ID /bin/bash`

## Featured results

- [Figure 3](#Figure-3)
- [Figure 4-a](#Figure-4-a)
- [Figure 5-a](#Figure-5-a)
- [Figure 5-b](#Figure-5-b)
- [Figure 6](#Figure-6)
- [Figure 7](#Figure-7)
- [Figure 8](#Figure-8)

Each experiment runs 5 to 100% load, in 5% increment. Each load point represents 20 seconds of traffic. Each figure presents 3 to 4 scheduling policies. Overall, we estimate that generating the results takes between 100 and 130mn.
Plotting itself takes time --- we are working on enhancing the data parsers to make them more performant.

The overall process could take between 3 and 4 hours.

## Notes

- Each figure has a list of experiment (one per load point). You can find this list in /psp/experiments/[Figure number].
- Though automated, some experiments could fail and you will have to rerun them. Usually, this means that a specific datapoint must be manually replayed.
- Specifically, we have seen cases where:
    - The server program does not stop and must be manually terminated (impact: Shremote hangs)
    - Shinjuku crashes due to "dune: exit due to unhandled VM exit"
    - Shremote fails to execute some management command over SSH such as removing a directory
- When plotting, if you see a message such as "/psp/experiments/CFCFS_0.70_DISP2_14.0/client0/traces_rates does not exist", you will have to run again this data point
- The run.py script can take a --load-range argument. Usage: --load-range low high, where low and high are load bounds used to generate 5% load increments per experiment. For example, --load-range .40 .41 will only run the experiment at 40% load.
- We are aware of an issue in terminating shinjuku that causes an error message to be printed in the shremote log, but does not impact the results.

In [1]:
%matplotlib widget
# %matplotlib notebook
import sys
import seaborn as sns
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import ScalarFormatter
from matplotlib import lines
import pandas as pd
import numpy as np
from pathlib import Path
from loader import *

# Figure 3

In [ ]:
# To generate the results, issue the following commands
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) Then run the following in the container:
# /psp/Shremote_cfgs/run.py 0 psp DISP2 -p DARC CFCFS DFCFS
plot_p99s(['Figure3'], app='MB', reset_cache=False, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True, add_shen=False, ncols=3)

# Figure 4-a

In [6]:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) Then run the following in the container:
# /psp/Shremote_cfgs/fig4.sh
plot_wcc('Figure4_a', reset_cache=False, value='p99', darc_cores=1, clients=[0,1,2,3,4,5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Figure 5-a

In [2]:
# Shinjuku data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh shinjuku` on the server
# 2) Then run the following in the container:
# /psp/Shremote_cfgs/run.py 0 shinjuku DISP2_IX -p cPREMQ --load-range .05 .80

# Perséphone data: already generated for figure 3

# Shenango data:
# 1) Reboot the machine with Linux 4.15 using pick_kernel.sh
# 2) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Shenango` on the server
# 3) /psp/Shremote_cfgs/run.py 0 shenango DISP2 -p CFCFS DFCFS --load-range .05 .80
plot_p99s(['Figure5_a'], app='MB', reset_cache=True, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/psp/experiments/shen-CFCFS_0.45_DISP2_14.0/client4/traces_rates does not exist
/psp/experiments/shen-CFCFS_0.45_DISP2_14.0/client4/traces_hist does not exist
[shen-CFCFS_0.45_DISP2_14.0] Missing 1 client histogram(s)
[Figure5_a] Parsed histograms in 261.639959 seconds


findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


[Figure5_a] Prepared df rows in 0.292466 seconds
[Figure5_a] Created df in 0.024886 seconds


# Figure 5-b

In [9]:
# Shinjuku data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh shinjuku` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 shinjuku SBIM2_IX -p cPREMQ --load-range .05 .60`

# Perséphone data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 psp SBIM2 -p DARC`

# Shenango data:
# 1) Reboot the machine with Linux 4.15 using pick_kernel.sh
# 2) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Shenango` on the server
# 3) `/psp/Shremote_cfgs/run.py 0 shenango DISP2 -p CFCFS DFCFS`

%pdb on
plot_p99s(['Figure5_b'], app='MB', reset_cache=True, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True)

Automatic pdb calling has been turned ON


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Figure5_b] Parsed histograms in 264.827361 seconds
[Figure5_b] Prepared df rows in 0.161678 seconds
[Figure5_b] Created df in 0.019760 seconds


# Figure 6

In [ ]:
# Shinjuku data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh shinjuku` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 shinjuku SBIM2_IX -p cPREMQ --load-range .05 .90`

# Perséphone data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 psp TPCC -p DARC`

# Shenango data:
# 1) Reboot the machine with Linux 4.15 using pick_kernel.sh
# 2) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Shenango` on the server
# 3) `/psp/Shremote_cfgs/run.py 0 shenango TPCC -p CFCFS DFCFS`
plot_p99s(['Figure6'], app='MB', ncols=-1, reset_cache=False, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True)

# Figure 7

In [8]:
# Shinjuku data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh shinjuku` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 shinjuku SBIM2_IX -p ROCKSDB_IX --load-range .05 .80 --app-type ROCKSDB`

# Perséphone data:
# 1) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Persephone` on the server
# 2) `/psp/Shremote_cfgs/run.py 0 psp ROCKSDB -p DARC --app-type ROCKSDB`

# Shenango data:
# 1) Reboot the machine with Linux 4.15 using pick_kernel.sh
# 2) `${AE_DIR}/Persephone/sosp_aec/base_start.sh Shenango` on the server
# 3) `/psp/Shremote_cfgs/run.py 0 shenango ROCKSDB -p CFCFS DFCFS --app-type ROCKSDB`

In [1]:
plot_p99s(['Figure7'], app='ROCKSDB', ncols=1, reset_cache=False, use_ylim=True, clients=[0,1,2,3,4,5], value="p99.9", close_all=True, remove_drops=True)

NameError: name 'plot_p99s' is not defined

# Figure 8

In [25]:
exps = ['DYN-RESA_0.80_sched4_14.40']
plot_agg_p99_over_time(exps, reset_cache=False, debug=False)

Automatic pdb calling has been turned ON


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

filling between 0.0 and 5.11282762742
filling between 5.11282762742 and 10.325906776947
filling between 10.325906776947 and 15.538985926473
filling between 15.538985926473 and 20.752065076
plotted data in 34.255152463912964
